# Gawler Challenge - Geochemistry Assay Pivot Tool

Download link for the notebook file - [DOWNLOAD](https://drive.google.com/open?id=1aAWabeI4dPCN-FC_Ln-o_fkA2RhjtJ7q)

Download link for the pivoted csv files - [DOWNLOAD](https://drive.google.com/open?id=1ETZsx4NQMmV3yATzkevrIIVeGj9YeAaQ)

This jupyter notebook contains a tool to convert the 10GB geochemistry sample file, **sarig_rs_chem_exp.csv**, into smaller csv's and shapefiles so assays such as drillcore, soil and rock chip can be easily visualised in GIS and other modelling software. Using python 3.7 and the libraries listed below, the tool contains 5 steps:

- **Step 1** - Importing libraries and ensuring they are correctly installed
- **Step 2** - Splitting the large csv into smaller sample based files
- **Step 3** - Splitting larger files into chunks
- **Step 4** - Pivoting each sample file to contain elemental columns
- **Step 5** - Convert each pivoted file to shapefile

The notebook runs on both Mac and Windows, takes approximately 5hr and has an output size of ~33GB total (unwanted files can be removed).

<img src="Images/step.png" />

### STEP 1 - Importing Libraries

If you've just downloaded Anaconda/Jupyter Notebook and getting stuck into Python, I'd suggest [this](https://towardsdatascience.com/beginners-guide-to-jupyter-notebook-8bb85b85085) good beginners guide for a read.

This is the first cell that needs to be run in the notebook, which imports all necessary libraries.

In [ ]:
#Importing required packages
import pandas as pd
import numpy as np
import glob
from geopandas import GeoDataFrame
from shapely.geometry import Point

If you get a **ModuleNotFoundError** error it is because the package python is trying to import is not installed. To change this, go back to Anaconda Navigator, click on Environments and search up the package ensuring that viewing dropdown is set to 'All'. Select the uninstalled package and the 'Apply' button will appear at the bottom of the window, click this to install. Once all the above packages have been installed and the cell above can run without error, proceed to step 2.

<img src="Images/anaconda.png" />

### STEP 2 - Splitting to CSV files

This step looks at breaking up the massive 10GB csv file into smaller more workable chunks based on their sample source (Drill Cuttings, Soil Sample, Outcrop Sample etc). To do this you must enter the file path to the csv file and also a file path to where you would like the newly made files to go. Here is an example;

- **filepath_to_chem = _'/Users/jackmaughan/desktop/SARIG_Data_Package/sarig_rs_chem_exp.csv'_**
- **filepath_to_out = _'/Volumes/External_Hardrive/Geochem_Samples/'_**

As the total output for all these files is ~33GB I would reccommend saving them to an external drive. Any unwanted files can obviously be deleted afterwards.

<img src="Images/step2.png" />

Set the desired filepaths below and click run on the cell (the play button in the toolbar).

In [ ]:
#SET THE FILEPATHS HERE
filepath_to_chem = '>>>INSERT FILEPATH HERE<<<'
filepath_to_out = '>>>INSERT FILEPATH HERE<<<'

There are no more inputs required in this notebook so you can run all the following cells below if desired. To do this click cell in the toolbar and press run all.

The next cell splits the csv file based on sample. Please note df stands for DataFrame. While the following cells run, you will see DtypeWarnings popping up, these can be ignored.

In [ ]:
#Reading the Geochem csv file as a pandas DataFrame
df = pd.read_csv(filepath_to_chem)

#Removing null values to unknown sample source to keep all values
df['SAMPLE_SOURCE'] = df['SAMPLE_SOURCE'].fillna('Unknown sample source')
df['SAMPLE_SOURCE_CODE'] = df['SAMPLE_SOURCE_CODE'].fillna('UNKNOWN')

#Removing two unwanted LOI interval values from Drill Cuttings file
df = df[~(df['VALUE'].str.contains('-') & df['VALUE'].str.contains('<'))]

#Saving individual sample types as csv files
for sample_type, group in df.groupby('SAMPLE_SOURCE_CODE'):
    group.to_csv(filepath_to_out+f'df_{sample_type}.csv', index=False)

#Resetting the DataFrame to remove some memory
df = None

Below is a table of what each sample code represents.

<img src="Images/SampleCode.png" />

### STEP 3 - Splitting larger files into chunks

Three of these output csv files (drill cuttings, drill core and half core) are still too large to work with in pandas. We could look at using [Dask](https://dask.org/) but for the sake of pandas simplicity we will just split these files up into four different chunks and save them. This step can be run without any changes required.

In [ ]:
#Adding the filenames to the bigger files
ct_filepath = filepath_to_out+'df_CT.csv'
dc_filepath = filepath_to_out+'df_DC.csv'
hcore_filepath = filepath_to_out+'df_HCORE.csv'

#Reading the files as pandas DataFrames
df_ct = pd.read_csv(ct_filepath)
df_dc = pd.read_csv(dc_filepath)
df_hcore = pd.read_csv(hcore_filepath)

#Setting number of splits
number_of_chunks = 4

#Splitting and saving files to output path
[df_i.to_csv(ct_filepath[0:-4]+'_{id}.csv'.format(id=id), index=False) for id, df_i in  enumerate(np.array_split(df_ct, number_of_chunks))]
[df_i.to_csv(dc_filepath[0:-4]+'_{id}.csv'.format(id=id), index=False) for id, df_i in  enumerate(np.array_split(df_dc, number_of_chunks))]
[df_i.to_csv(hcore_filepath[0:-4]+'_{id}.csv'.format(id=id), index=False) for id, df_i in  enumerate(np.array_split(df_hcore, number_of_chunks))]

### STEP 4 - Pivoting Table

This step is the most 'pivotal' (⌐■_■) in regards to wanting to visualise our data. This step pivots the 'CHEM CODE' column into separate columns based on the elemental value, while converting all the values into ppm for easy comparison and retaining all the original lithology and spatial data. This transformation is represented visually below.

<img src="Images/pivoted.png" />

NOTE: In this step;
 - All values have been converted to ppm
 - Any unique units that can't be converted to ppm (cps and us/cm) and unknown units have been removed
 - You will notice that each sample may have different Assay methods (XRF, ICP-MS etc)
 - Any assay that has a 'below detection limit' result is represented as a negative value. The value indicates the detection limit in ppm (ie if a value has a detection limit of <0.02ppm this will now be represented as -0.02ppm).
 
There are no inputs required for this cell and it can be run straight away.

In [ ]:
#Loading in all the csv files
files = glob.glob(filepath_to_out+'*.csv')
files = [i.replace('\\', '/') for i in files]

#Removing the three bigger files
files.remove(ct_filepath)
files.remove(dc_filepath)
files.remove(hcore_filepath)

#Setting pivot to loop over all files
for file in files:
    df = pd.read_csv(file)
    
    #Removing rows with unwanted units (cps: radiometrics, NOUNIT: pH values, us/cm:electrical conductivity, X: ?)
    df = df[(df['UNIT'] != 'cps') & (df['UNIT'] != 'NOUNIT') & (df['UNIT'] != 'us/cm') & (df['UNIT'] != 'X')]
    
    #Removing any string values in the assay column
    df['VALUE'] = df['VALUE'].astype(str)
    df['VALUE'] = df['VALUE'].str.replace('<', '-').str.replace('>', '-').astype(float)

    #Converting all assay to ppm
    df.loc[df['UNIT'] == 'ppb', 'VALUE_PPM'] = df['VALUE']/1000
    df.loc[df['UNIT'] == 'ppm', 'VALUE_PPM'] = df['VALUE']
    df.loc[df['UNIT'] == '%', 'VALUE_PPM'] = df['VALUE']*10000
    df.loc[df['UNIT'] == 'g/T', 'VALUE_PPM'] = df['VALUE']
    df.loc[df['UNIT'] == 'u/L', 'VALUE_PPM'] = df['VALUE']/1000
    df.loc[df['UNIT'] == 'mg/L', 'VALUE_PPM'] = df['VALUE']
    
    #Filling null CHEM_METHOD_CODE so they don't get removed from our data
    df['CHEM_METHOD_CODE'] = df['CHEM_METHOD_CODE'].fillna('unknown')
    
    #Pivoting the table and renaming the axis
    df_p = pd.pivot_table(df, index=['SAMPLE_NO', 'CHEM_METHOD_CODE'], columns='CHEM_CODE', values='VALUE_PPM')
    df_p = df_p.rename_axis(None, axis=1).reset_index()
    
    #Removing redundant columns
    df=df.drop(['CHEM_CODE', 'VALUE', 'UNIT', 'CHEM_METHOD_CODE',
                'CHEM_METHOD_DESC', 'VALUE_PPM', 'SAMPLE_ANALYSIS_NO',
                'OTHER_ANALYSIS_ID'], axis=1)
    
    #Merging with old file to retain spatial data
    df_all = df_p.merge(df, how='left', left_on='SAMPLE_NO', right_on='SAMPLE_NO').drop_duplicates()
    
    #Saving all CSV files to location
    df_all.to_csv(file[0:-4]+'_Pivoted.csv', index = False)

If you wish to free up some room on your drive (and don't want to keep  them) the csv files that **don't** contain the word 'Pivoted' can now be removed. These are manually removed by going into your output folder and deleting them.

### STEP 5 - Converting to Shapefile

This final step takes those pivoted csv files and converts them to GIS friendly shapefiles. They are set to the GDA2020 CRS (lat-long) and can be dropped straight into a QGIS project for further reprojection if necessary. Keep in mind there are 4 different shapefiles created for Drill Cuttings, Drill Core and Half Core. 

In [ ]:
#Rereading pivoted csv files
files = glob.glob(filepath_to_out+'*_Pivoted.csv')

#Creating a loop for all files
for file in files:
    
    #reading in the files
    df = pd.read_csv(file)
    
    #Setting the geometry and crs for the files
    geometry = [Point(xy) for xy in zip(df['LONGITUDE_GDA2020'], df['LATITUDE_GDA2020'])]
    crs = {'init': 'epsg:7844'}
    
    #Creating a geopandas DataFrame from the spatial data
    gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
   
    #Saving the shapefiles to file
    gdf.to_file(file[0:-4]+'.shp')

The image below is an example of the final product, outcrop samples with respective Cu values shown in QGIS. I hope this notebook is of use and can help analyse geochemistry data.

<img src="Images/example.png" />